In [2]:
import re

from joblib import load

from utils.preprocessing import preprocessing

order_labels = ['B-Quantity', 'B-Pizza', 'I-Pizza', 'B-Topping', 'B-Size', 'I-Size', 'O', 'B-Crust', 'I-Crust']
customer_info_labels = ['B-Cus', 'I-Cus', 'B-Address', "I-Address", 'B-Phone', 'O', 'B-Payment', 'I-Payment']


class EntitiesRecognizer:
    def __init__(self, model_path: str, is_order: bool):
        self.model = self._load_model(model_path)
        self.labels = order_labels if is_order else customer_info_labels

    def _load_model(self, model_path):
        return load(model_path)

    def word2features(self, sentence, i):
        word = sentence[i]
        features = {
            "bias": 1.0,
            "word.lower()": word.lower(),
            "word[-3:]": word[-3:],
            "word[-2:]": word[-2:],
            "word.isupper()": word.isupper(),
            "word.isdigit()": word.isdigit(),
        }
        if i > 0:
            word1 = sentence[i - 1]
            features.update(
                {
                    "-1:word.lower()": word1.lower(),
                    "-1:word.isupper()": word1.isupper(),
                    "-1:word.isdigit()": word1.isdigit(),
                }
            )
        else:
            features["BOS"] = True

        if i < len(sentence) - 1:
            word1 = sentence[i + 1]
            features.update(
                {
                    "+1:word.lower()": word1.lower(),
                    "+1:word.isupper()": word1.isupper(),
                    "+1:word.isdigit()": word1.isdigit(),
                }
            )
        else:
            features["EOS"] = True

        return features

    def sentence_features(self, words):
        return [self.word2features(words, i) for i in range(len(words))]

    def sentence_labels(self, labels):
        return labels

    def process_sentence(self, sentence):
        tokens = re.findall(r"[\w']+|[.,!?;]", sentence)
        return {
            "words": tokens,
            "label": ["O"] * len(tokens),
        }

    def predict(self, text):
        text = preprocessing(text, True)
        processed_sentence = self.process_sentence(text)
        words = processed_sentence["words"]
        features = self.sentence_features(words)

        labels = self.model.predict([features])[0]

        result = {
            "words": words,
            "label": labels,
        }
        return self.aggregate_entities(result)

    def aggregate_entities(self, predicted_result):
        aggregated_entities = {}
        current_entity = None
        current_label = None

        for word, label in zip(predicted_result["words"], predicted_result["label"]):
            if label.startswith("B-"):
                if current_entity is not None and current_label is not None:
                    if current_label in aggregated_entities:
                        aggregated_entities[current_label].append(
                            " ".join(current_entity)
                        )
                    else:
                        aggregated_entities[current_label] = [" ".join(current_entity)]

                current_entity = [word]
                current_label = label[2:]
            elif (
                label.startswith("I-")
                and current_entity is not None
                and label[2:] == current_label
            ):
                current_entity.append(word)
            else:
                if current_entity is not None and current_label is not None:
                    if current_label in aggregated_entities:
                        aggregated_entities[current_label].append(
                            " ".join(current_entity)
                        )
                    else:
                        aggregated_entities[current_label] = [" ".join(current_entity)]
                    current_entity = None
                    current_label = None
                if label == "O":
                    continue
                else:
                    aggregated_entities[label] = aggregated_entities.get(label, []) + [
                        word
                    ]

        if current_entity is not None and current_label is not None:
            if current_label in aggregated_entities:
                aggregated_entities[current_label].append(" ".join(current_entity))
            else:
                aggregated_entities[current_label] = [" ".join(current_entity)]

        return aggregated_entities


In [4]:
order_model = EntitiesRecognizer("../output/savedmodels/order_entity_v2.h5", True)
customer_info_model = EntitiesRecognizer("../output/savedmodels/customer_info_entity_v1.h5", False)

In [18]:
order_model.predict("e cho t xin 2 cai pepperoni voi nam rom va ba chi bo nuong")

{'Quantity': ['2'],
 'Pizza': ['cai pepperoni'],
 'Topping': ['nấm_rơm', 'ba_chỉ_bò_nướng']}

In [19]:
customer_info_model.predict("ship den pho thanh xuan cho t")

{'Address': ['thanh_xuân']}

In [16]:
import torch
import torch.nn as nn
import numpy as np
from transformers import AutoModel, AutoTokenizer
from utils.preprocessing import preprocessing

THRESHOLD = 0.99
MAX_LEN = 128
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


class IntentsRecognizer(nn.Module):
    def __init__(self):
        super(IntentsRecognizer, self).__init__()
        self.phobert = AutoModel.from_pretrained("vinai/phobert-base")
        self.dropout = nn.Dropout(p=0.3)
        self.linear = nn.Linear(768, 9)
        self.intent_labels = [
            "view_menu",
            "view_cart",
            "add_to_cart",
            "remove_from_cart",
            "modify_cart_item",
            "confirm_order",
            "track_order",
            "cancel_order",
            "provide_info",
        ]
        self.intent_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.phobert(input_ids, attention_mask=attn_mask, token_type_ids=token_type_ids)
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

    def predict(self, text):
        text = preprocessing(text, False)
        encoded_text = self.intent_tokenizer.encode_plus(
            text,
            max_length=MAX_LEN,
            add_special_tokens=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors="pt",
        )
        input_ids = encoded_text["input_ids"].to(device)
        attention_mask = encoded_text["attention_mask"].to(device)
        token_type_ids = encoded_text["token_type_ids"].to(device)
        output = self(input_ids, attention_mask, token_type_ids)
        probabilities = torch.softmax(output, dim=-1).detach().cpu().numpy().flatten()

        max_prob = np.max(probabilities)
        max_prob_index = np.argmax(probabilities)

        print(probabilities)
        if max_prob >= THRESHOLD:
            return self.intent_labels[max_prob_index]
        else:
            return None

In [17]:
intent_model = IntentsRecognizer()
intent_model.load_state_dict(torch.load("../output/savedmodels/intents_v2.bin"))
intent_model.to(device)

IntentsRecognizer(
  (phobert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [27]:
intent_model.predict("e xoa 1 cai pepperoni na")

[5.91382050e-05 1.05170882e-04 1.97371322e-04 9.99201596e-01
 1.00157326e-04 7.44513309e-05 8.43253147e-05 1.04384693e-04
 7.33972411e-05]


'remove_from_cart'